### Article Predictions 

#### We made a article recommendation system to predict the articles based on user intrests which is given in the form of prompt

In [2]:
import pandas as pd


column_names = ['Item Title', 'Publication Title', 'Item DOI', 'Authors', 'Publication Year', 'URL', 'Keywords', 'Language']

main = pd.read_csv('output.csv',encoding="utf-8",header=None, names=column_names)
main=main.drop(['Language','Item DOI'],axis=1)


c:\Users\vukya\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\vukya\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\vukya\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Journal Dataframe

#### Journal articles, keywords, authors concating them

In [3]:
journal_art = main.groupby('Publication Title')['Item Title'].apply(list).reset_index(name='Articles')
journal_art.set_index(['Publication Title'],inplace=True)
journal_auth = main.groupby('Publication Title')['Authors'].apply(list).reset_index(name='Authors')
journal_auth.set_index(['Publication Title'],inplace=True)
journal_key= main.drop_duplicates(subset=["Publication Title", "Keywords"], keep='first')
journal_key=journal_key.drop(['Item Title','Authors','Publication Year','URL'],axis=1)
journal_key.set_index(['Publication Title'],inplace=True)
journal_main = pd.concat([journal_key, journal_art,journal_auth], axis=1, join='inner')
journal_main=journal_main.reset_index()
journal_main.head()

,Publication Title,Keywords,Articles,Authors
0,AAPPS Bulletin,AAPPS Bulletin Atomic Molecular Optical and Pl...,[Correction to Strong field physics pursued wi...,[Vishwa Bandhu PathakSeong Ku LeeKi Hong PaeCa...
1,AAPS PharmSciTech,AAPS PharmSciTech PharmacologyToxicology Biote...,[Correction Environmental Monitoring for Close...,[Joseph McCallNonita BarnardKevin GadientChand...
2,AI SOCIETY,NaN,[Correction to Dismantling the Chinese Room wi...,"[Lawrence Lengbeyer, Clare L E Foster, Frances..."
3,AI and Ethics,NaN,[Publisher Correction Convergence of the sourc...,"[Haleh Asgarinia, Jakob MkanderLuciano Floridi..."
4,AI in Civil Engineering,NaN,"[AI in Civil Engineering, Fusion of thermal an...","[Xianzhong Zhao, Quincy G AlexanderVedhus Hosk..."


#### Changing into paragraphs from lists

##### get paragraph:
1. combines joins the list of the data into a paragraph
##### get cleaned_text:
1. check if the instance is string
2. re.match(r'^[a-zA-Z]+$', word) checks if the word consists only of alphabetic characters
3. word not in stop_words checks if the word is not in a list of stop words.
4. len(word) > 1 checks if the word has a length greater than 1.
5. word[1] != '.' checks if the second character of the word is not a dot (.) character.
##### combine text:
1. combines all the indices required in the list of indices into a paragraph

In [4]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def get_paragraph(row,index):
  ans=''
  for x in row[index]:
    ans=ans+'  '+x.lower()
  return ans
def get_clean_text(row,index):
    if not isinstance(row[index], str):
        return '' 
    clean_text=''
    words = re.findall(r'\b\w+\b', row[index].lower())
    words = row[index].lower().split()
    for word in words:
        if( re.match(r'^[a-zA-Z]+$', word) and  word not in stop_words and len(word) >1 and word[1] != '.'):
            clean_text=clean_text+' '+word
    return clean_text
def combine(row,indices):
    ans=''
    for i in indices:
      ans=ans+' '+row[i]
    return ans

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vukya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vukya\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vukya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
journal_main['Articles']=journal_main.apply(get_paragraph,index='Articles',axis=1)
journal_main['Articles']=journal_main.apply(get_clean_text,index='Articles',axis=1)
journal_main['Authors']=journal_main.apply(get_paragraph,index='Authors',axis=1)
journal_main['Authors']=journal_main.apply(get_clean_text,index='Authors',axis=1)
journal_main['Keywords']=journal_main.apply(get_clean_text,index=1,axis=1)
journal_main.head()

,Publication Title,Keywords,Articles,Authors
0,AAPPS Bulletin,aapps bulletin atomic molecular optical plasm...,correction strong field physics pursued petaw...,vishwa bandhu pathakseong ku leeki hong paeca...
1,AAPS PharmSciTech,aaps pharmscitech pharmacologytoxicology biot...,correction environmental monitoring closed ro...,joseph mccallnonita barnardkevin gadientchand...
2,AI SOCIETY,,correction dismantling chinese room linguisti...,lawrence lengbeyer clare foster francesca fof...
3,AI and Ethics,,publisher correction convergence source contr...,haleh asgarinia jakob mkanderluciano floridi ...
4,AI in Civil Engineering,,ai civil engineering fusion thermal rgb image...,xianzhong zhao quincy alexandervedhus hoskere...


### Feature Extraction taking nouns and adjectives as features

#### Preprocessing the text

It is possible to predict journals using only nouns and adjectives as features. By focusing on nouns and adjectives, we can capture key information and characteristics of the articles that may be relevant for predicting the corresponding journals.

In [6]:
journal_main['Tokenized'] = journal_main['Articles'].apply(word_tokenize)
journal_main['Tagged'] = journal_main['Tokenized'].apply(pos_tag)

In [7]:
journal_main['Tags'] = journal_main['Tagged'].apply(lambda x: [word for word, tag in x if tag.startswith('NN') or tag.startswith('JJ')])
journal_main['Tags'] = journal_main['Tags'].apply(lambda x: [word for word in x if word.lower() not in stop_words])
journal_main['Tags']=journal_main.apply(get_paragraph,index='Tags',axis=1)
journal_main=journal_main.drop(['Articles','Tokenized','Tagged'],axis=1)
journal_main['Tags']=journal_main.apply(combine,indices=['Keywords','Tags','Authors'],axis=1)
journal_main['Tags']=journal_main.apply(get_clean_text,index='Tags',axis=1)
journal_main=journal_main.drop(['Keywords','Authors'],axis=1)
journal_main.head()

,Publication Title,Tags
0,AAPPS Bulletin,aapps bulletin atomic molecular optical plasm...
1,AAPS PharmSciTech,aaps pharmscitech pharmacologytoxicology biot...
2,AI SOCIETY,correction chinese room linguistic tools conc...
3,AI and Ethics,publisher correction convergence source contr...
4,AI in Civil Engineering,civil engineering fusion thermal rgb images d...


### TFIDF vector for journal prediction 

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer(decode_error='ignore',strip_accents='ascii')
journal_tfidf_matrix = vectorizer.fit_transform(journal_main['Tags'])

we predict based on cosine similarities

In [1]:
journal_threshold=4

In [10]:
def get_journal_index(user_input):
    user_tfidf = vectorizer.transform([user_input])
    cosine_similarities = cosine_similarity(user_tfidf, journal_tfidf_matrix).flatten()
    indices = cosine_similarities.argsort()[::-1]
    top_recommendations = [i for i in indices if cosine_similarities[i] > 0][:min(journal_threshold, len(indices))]
    return top_recommendations

#### Preparing individual article dataframes

In [11]:
def get_article_df(row):
    article = main.loc[main['Publication Title'] == journal_main['Publication Title'][row.name]].copy()
    article['Item Title']=article.apply(get_clean_text,index='Item Title',axis=1)
    article['Authors']=article.apply(get_clean_text,index='Authors',axis=1)
    article['Tokenized'] = article['Item Title'].apply(word_tokenize)
    article['Tagged'] = article['Tokenized'].apply(pos_tag)
    article['Tags'] = article['Tagged'].apply(lambda x: [word for word, tag in x if tag.startswith('NN') or tag.startswith('JJ')and word.lower() not in stop_words])
    article['Tags']=article.apply(get_paragraph,index='Tags',axis=1)
    article['Tags']=article.apply(lambda x : x['Tags']+' '+x['Authors']+' '+str(x['Publication Year']),axis=1)
    article=article.drop(['Keywords','Publication Title','Tokenized','Tagged','Authors','Publication Year'],axis=1)
    article.reset_index(inplace=True)
    article.set_index('index', inplace=True)
    return article

journal_main['article_df']=journal_main.apply(get_article_df,axis=1)

#### Generating tfidf matrices for the journals

In [12]:
def get_vectorizer(row):
    vectorizer = TfidfVectorizer(decode_error='ignore',strip_accents='ascii')
    return vectorizer
def get_tfidf_matrix(row):
    tfidf_matrix = row['article_vectorizer'].fit_transform(row['article_df']['Tags'])
    return tfidf_matrix
journal_main['article_vectorizer']=journal_main.apply(get_vectorizer,axis=1)
journal_main['article_matrix']=journal_main.apply(get_tfidf_matrix,axis=1)

In [13]:
article_threshold=10

#### Article recommendation using user input

In [14]:
def get_article_recommendations(user_input):
    recommended_journals=get_journal_index(user_input)
    l=[]
    for journal_id in recommended_journals:
        user_tfidf=journal_main['article_vectorizer'][journal_id].transform([user_input])
        cosine_similarities = cosine_similarity(user_tfidf, journal_main['article_matrix'][journal_id]).flatten()
        indices = cosine_similarities.argsort()[::-1]
        top_recommendation_articles = [(cosine_similarities[i],i,journal_id) for i in indices if cosine_similarities[i] > 0][:min(article_threshold, len(indices))]
        l=l+top_recommendation_articles
        print(journal_id)
    l.sort(reverse=True)
    return l


In [15]:

def get_links(user_input):
    l=[]
    recommendation_list=get_article_recommendations(user_input) #You will get article id and journal id from here
    for article in recommendation_list:
        cosine_similarity,article_id,journal_id=article
        l.append((journal_main['article_df'][journal_id].iloc[article_id,0],journal_main['article_df'][journal_id].iloc[article_id,1],article_id,journal_id))
        # print(name,url)
        print()
    return l
user_input = "AI is my favourite language in 2022"
l=get_links(user_input)
print(l)


[(' language test activism', 'http://link.springer.com/article/10.1007/s10993-022-09614-7', 25, 1524), (' attribution autonomy role robotic language acquisition', 'http://link.springer.com/article/10.1007/s00146-020-01114-8', 196, 2), (' language advocacy times securitization neoliberalization network languagerights', 'http://link.springer.com/article/10.1007/s10993-022-09617-4', 31, 1524), (' bias comparison framework abusive language datasets', 'http://link.springer.com/article/10.1007/s43681-021-00081-0', 119, 3), (' auditing large language models threelayered approach', 'http://link.springer.com/article/10.1007/s43681-023-00289-2', 67, 3), (' officiality strategic ambiguity language policy exploring migrant experiences andorra luxembourg', 'http://link.springer.com/article/10.1007/s10993-021-09602-3', 29, 1524), (' identity ai', 'http://link.springer.com/article/10.1007/s44163-022-00038-0', 59, 599), (' evolution language ideological debates english french multilingual humanitarian

In [16]:
import joblib
joblib.dump(journal_main,'journal_df.joblib')

['journal_df.joblib']

In [17]:
joblib.dump(journal_tfidf_matrix,'journal_tfidf.joblib')

['journal_tfidf.joblib']

In [18]:
joblib.dump(vectorizer,'journal_vectorizer.joblib')

['journal_vectorizer.joblib']

In [21]:
main.describe()

,Publication Year
count,279624.000000
mean,2018.662740
std,4.398261
min,1971.000000
25%,2017.000000
50%,2020.000000
75%,2022.000000
max,2023.000000
